In [1]:
import pandas as pd
import numpy as np

In [18]:

sim_hist = pd.DataFrame()
for sim in range(10000):
    #init condition setup
    df = [['A',0,1],['B',0,2],['C',0,3],['D',0,4],['E',0,5]]
    Camel_list = pd.DataFrame(df,columns=['name','location','order'])
    Camel_list['moved'] = 0
    big_run = 0
    Camel_in_dice = range(len(Camel_list))
    run_hist = pd.DataFrame()
    small_game_hist = pd.DataFrame()
    

    while Camel_list['location'].max()<=16:
        big_run = big_run+1
        for run_th,Camel_a_run in enumerate(range(len(Camel_list))):
            # random pick A~E
            selected_camel_i = np.random.choice(Camel_in_dice)
            selected_camel = Camel_list.iloc[selected_camel_i]
            # selected, roll dice 1,2,3
            Camel_move = np.random.choice(range(3))+1
            
            # update location
            Camel_list['order_update'] = Camel_list.apply(lambda x: 1 if (x['location'] == selected_camel.location) and  (x['order'] >= selected_camel.order) 
                                                else 0,axis=1)
            Camel_list.location = Camel_list.apply(lambda x: x['location'] + Camel_move if x['order_update'] else x['location'],axis=1)

            #update the order
            Camel_list['order'] = Camel_list['order'] + 10 * Camel_list['order_update'] 
            Camel_list['order'] = Camel_list.groupby("location")["order"].rank(method="dense")
            
            #update for next run
            Camel_in_dice = [i for i in Camel_in_dice if i !=selected_camel_i ]
            Camel_list['moved'] = Camel_list['moved'] +Camel_list['order_update']
            Camel_list['order_update'] = 0

            #history record
            run_record = {'run':run_th,'name':selected_camel['name'],'move':Camel_move,}
            run_hist = pd.concat([run_hist,pd.DataFrame(run_record,index=['%s'  % (run_th)])])
            #display(run_record)
            #display(Camel_list)

            #detect if game end
            if Camel_list['location'].max()>16: break
            
        # a small run record
        small_game_record = pd.DataFrame(Camel_list)
        small_game_record['rank'] = (small_game_record['location'] * 10 +small_game_record['order'] ).rank(method="dense",ascending = False)
        small_game_record['big_run'] = big_run
        small_game_hist = pd.concat([small_game_hist,small_game_record])

        #reset the random pick
        Camel_in_dice = range(len(Camel_list))
        
    #simulate record
    Camel_list_filter = Camel_list[(Camel_list['location'] ==Camel_list['location'].max())]
    Camel_list_filter = Camel_list_filter[(Camel_list_filter['order'] ==Camel_list_filter['order'].max())]
    winner = Camel_list_filter['name'].iloc[0]

    Camel_list_filter = Camel_list[(Camel_list['location'] ==Camel_list['location'].min())]
    Camel_list_filter = Camel_list_filter[(Camel_list_filter['order'] ==Camel_list_filter['order'].min())]
    loser = Camel_list_filter['name'].iloc[0]

    pv_record = pd.pivot_table(small_game_hist,index='name',columns='rank',aggfunc='count',fill_value=0)['big_run']
    sim_record = {'winner': winner,
                  'last lose':loser,
                  'A first count':pv_record.loc['A',1],
                  'B first count':pv_record.loc['B',1],
                  'C first count':pv_record.loc['C',1],
                  'D first count':pv_record.loc['D',1],
                  'E first count':pv_record.loc['E',1],
                  'A second count':pv_record.loc['A',2],
                  'B second count':pv_record.loc['B',2],
                  'C second count':pv_record.loc['C',2],
                  'D second count':pv_record.loc['D',2],
                  'E second count':pv_record.loc['E',2],
                  'A late count':pv_record.loc['A',5],
                  'B late count':pv_record.loc['B',5],
                  'C late count':pv_record.loc['C',5],
                  'D late count':pv_record.loc['D',5],
                  'E late count':pv_record.loc['E',5],
                  'max run': big_run
                  }
    sim_hist = pd.concat([sim_hist,pd.DataFrame(sim_record,index=['%s'  % (sim)])])


#display(Camel_list)
#display(run_hist)
#display(small_game_hist)
display(sim_hist)


,winner,last lose,A first count,B first count,C first count,D first count,E first count,A second count,B second count,C second count,D second count,E second count,A late count,B late count,C late count,D late count,E late count,max run
0,C,A,0,0,1,5,0,0,1,4,1,0,4,1,0,0,1,6
1,B,D,0,3,2,0,0,2,2,1,0,0,0,0,0,5,0,5
2,C,A,1,0,1,0,2,1,1,1,0,1,2,1,0,1,0,4
3,D,B,0,0,4,2,1,3,0,0,0,4,0,7,0,0,0,7
4,D,A,0,1,1,4,0,0,0,3,2,1,2,2,0,0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,C,E,2,0,2,1,0,3,0,1,1,0,0,3,0,0,2,5
9996,D,C,0,1,0,3,3,5,0,0,1,1,0,0,7,0,0,7
9997,A,C,4,1,0,0,0,1,4,0,0,0,0,0,4,1,0,5
9998,C,E,1,1,3,0,0,2,0,2,1,0,1,1,0,0,3,5


In [19]:
display(sim_hist.groupby('winner').count()['max run'])
display(sim_hist.groupby('last lose').count()['max run'])
sim_hist.describe().loc[['mean','std']]

winner
A    1651
B    1936
C    2023
D    2151
E    2239
Name: max run, dtype: int64

last lose
A    2917
B    2149
C    1783
D    1630
E    1521
Name: max run, dtype: int64

,A first count,B first count,C first count,D first count,E first count,A second count,B second count,C second count,D second count,E second count,A late count,B late count,C late count,D late count,E late count,max run
mean,0.745200,0.936600,1.051100,1.147700,1.34440,0.785700,0.933500,1.070600,1.237700,1.197500,1.680800,1.144000,0.899200,0.800900,0.70010,5.225000
std,1.074799,1.191773,1.249658,1.287187,1.39477,1.055408,1.112026,1.146101,1.221535,1.192659,1.815665,1.580606,1.446668,1.386453,1.28161,0.815378


In [26]:
small_game_hist = pd.DataFrame()
for sim in range(10000):
    #init condition setup
    df = [['A',0,1],['B',0,2],['C',0,3],['D',0,4],['E',0,5]]
    Camel_list = pd.DataFrame(df,columns=['name','location','order'])
    Camel_list['moved'] = 0
    big_run = 0
    Camel_in_dice = range(len(Camel_list))
    run_hist = pd.DataFrame()
    
    

    for run_th,Camel_a_run in enumerate(range(len(Camel_list))):
        # random pick A~E
        selected_camel_i = np.random.choice(Camel_in_dice)
        selected_camel = Camel_list.iloc[selected_camel_i]
        # selected, roll dice 1,2,3
        Camel_move = np.random.choice(range(3))+1
        
        # update location
        Camel_list['order_update'] = Camel_list.apply(lambda x: 1 if (x['location'] == selected_camel.location) and  (x['order'] >= selected_camel.order) 
                                            else 0,axis=1)
        Camel_list.location = Camel_list.apply(lambda x: x['location'] + Camel_move if x['order_update'] else x['location'],axis=1)

        #update the order
        Camel_list['order'] = Camel_list['order'] + 10 * Camel_list['order_update'] 
        Camel_list['order'] = Camel_list.groupby("location")["order"].rank(method="dense")
        
        #update for next run
        Camel_in_dice = [i for i in Camel_in_dice if i !=selected_camel_i ]
        Camel_list['moved'] = Camel_list['moved'] +Camel_list['order_update']
        Camel_list['order_update'] = 0

        #history record
        run_record = {'run':run_th,'name':selected_camel['name'],'move':Camel_move,}
        run_hist = pd.concat([run_hist,pd.DataFrame(run_record,index=['%s'  % (run_th)])])
        #display(run_record)
        #display(Camel_list)

        #detect if game end
        if Camel_list['location'].max()>16: break

    # a small run record
    small_game_record = pd.DataFrame(Camel_list)
    small_game_record['rank'] = (small_game_record['location'] * 10 +small_game_record['order'] ).rank(method="dense",ascending = False)
    small_game_record['big_run'] = big_run
    small_game_hist = pd.concat([small_game_hist,small_game_record])

    #reset the random pick
    Camel_in_dice = range(len(Camel_list))

pd.pivot_table(small_game_hist,index='name',columns='rank',aggfunc='count',fill_value=0)['big_run']


rank,1.0,2.0,3.0,4.0,5.0
name,,,,,
A,803,941,1558,2640,4058
B,1640,1692,2003,2544,2121
C,2021,2052,2560,1848,1519
D,2384,2867,1950,1553,1246
E,3152,2448,1929,1415,1056


In [ ]:
# code removed
# make propoty
class Camel():
    def __init__(self,name, location,order):
        self.name = name
        self.location = location
        self.order = order #1 is the most bottom

'''
Camel_list = [Camel("A",0,1),
Camel("B",0,2),
Camel("C",0,3),
Camel("D",0,4),
Camel("E",0,5),
]

'''